 Python script to generate production profile curves, potential production curves and drilling schedule
 The input data consists of:
 - Type curves (one well oil, gas and water production as function of time).
 - Initial reservoir volume (STOOIP) and recovery factor.
 - Drilling information: number of rigs, average drilling time, Producer-injector ratio, maximum number of wells
   allowed in the field.
 - Plateau rate.

In [20]:
# Import Python libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import math
import os

In [21]:
from IPython.core.display import display
from ipywidgets import widgets

DrillingSch_json_file = "DrillSch.json"
DrillingData_json_file = "DrillingData.json"
OtherInput_json_file = "OtherInput.json"
upload_drilling_sch = widgets.FileUpload(accept='.json', multiple=False, description="drilling schedule")
upload_drilling_data = widgets.FileUpload(accept='.json', multiple=False, description="drilling data")
upload_other_input = widgets.FileUpload(accept='.json', multiple=False, description="other")
#display(widgets.Label("upload_drilling_sch"), upload_drilling_sch)
display(upload_drilling_sch)
display(upload_drilling_data)
display(upload_other_input)


FileUpload(value={}, accept='.json', description='drilling schedule')

FileUpload(value={}, accept='.json', description='drilling data')

FileUpload(value={}, accept='.json', description='other')

In [ ]:
import ipysheet
from ipywidgets import widgets

In [4]:
########################################################################################################################
############                                                                                                ############
############                  Read input data and generate pandas dataframes from it                        ############
############                                                                                                ############
########################################################################################################################


# Get data from input files

# Create a list with the names of all the type curves files available

TypeCurves_names = []
for root, dirs, files in os.walk(os.getcwd()):
    for file_name in files:
        if file_name.startswith("TypeCurve"):
            TypeCurves_names.append(file_name)


# TypeCurve_json_file = 'TypeCurve.json'
DrillingSch_json_file = "DrillSch.json"
DrillingData_json_file = "DrillingData.json"
OtherInput_json_file = "OtherInput.json"

# Create pandas DataFrame from input data
TypeCurves_dict = {}
for element in TypeCurves_names:
    TypeCurves_dict[element.split(".")[0]] = pd.DataFrame(pd.read_json(element))

DrillingData_df = pd.DataFrame([pd.read_json(DrillingData_json_file, typ="series")])
OtherInput_df = pd.DataFrame(pd.read_json(OtherInput_json_file))
# TypeCurve_df = pd.DataFrame(pd.read_json(TypeCurve_json_file))
# DrillingSch_df = pd.read_json(DrillingSch_json_file)


# Declare dictionaries where the results for all the type curves are stored
DrillingSch_dict = {}
pot_prod_dict = {}
prod_profile_dict = {}
prod_profile_parameter_dict = {}

# Generate results for each type curves
n_file = 1
for key, val in TypeCurves_dict.items():
    TypeCurve_df = val

    # Format date strings to date object in "TypeCurve" input and add new column with time in days
    # change date string format to date object
    for date in range(1, len(TypeCurve_df)):
        TypeCurve_df.loc[date, "time"] = datetime.datetime.strptime(
            TypeCurve_df.loc[date, "time"], "%d-%m-%Y"
        ).date()

    # new column of time in date
    TypeCurve_df.insert(loc=1, column="time_days", value=0)

    for d in range(2, len(TypeCurve_df)):
        TypeCurve_df.loc[d, "time_days"] = (
            TypeCurve_df.loc[d - 1, "time_days"]
            + (TypeCurve_df.loc[d, "time"] - TypeCurve_df.loc[d - 1, "time"]).days
        )
    TypeCurve_df.loc[0, "time_days"] = "days"

    # print(TypeCurve_df)
    # print(DrillingData_df)
    # print(OtherInput_df)

ValueError: Expected object or value

In [ ]:
    ########################################################################################################################
    ############                                                                                                ############
    ############                  Calculate total recoverables and plateau rate                                 ############
    ############                                                                                                ############
    ########################################################################################################################

    # Estimate recoverable reserves, plateau rate and cumulative production during plateau period
    recoverables = OtherInput_df.loc[1, "RF"] * 0.01 * OtherInput_df.loc[1, "STOOIP"]
    # print (recoverables)

    if OtherInput_df.loc[1, "plateau_rate"] != 0:
        Plateau_rate = OtherInput_df.loc[1, "plateau_rate"] * 0.01 * recoverables

        parameters_dict = {
            "Recoverables Reserves": [
                str(OtherInput_df.loc[0, "STOOIP"]),
                recoverables,
            ],
            "Plateau Rate": [
                str(OtherInput_df.loc[0, "STOOIP"]) + "/days",
                Plateau_rate,
            ],
        }
    else:
        Plateau_rate = (
            OtherInput_df.loc[1, "plateau_cum"]
            * 0.01
            * recoverables
            / OtherInput_df.loc[1, "plateau_length"]
        )

        parameters_dict = {
            "Recoverables Reserves": [
                str(OtherInput_df.loc[0, "STOOIP"]),
                recoverables,
            ],
            "Plateau Rate": [
                str(OtherInput_df.loc[0, "STOOIP"]) + "/days",
                Plateau_rate,
            ],
        }

    # print(Plateau_rate)
    parameters = pd.DataFrame.from_dict(parameters_dict)
    # print(parameters)

    prod_profile_parameter_dict["prod_prod_param" + key.split("_")[1]] = parameters
    # print(prod_profile_parameter_dict)

In [ ]:
    ########################################################################################################################
    ############                                                                                                ############
    ############                       Estimate drilling schedule                                               ############
    ############                                                                                                ############
    ########################################################################################################################

    # Drilling Schedule
    Rigs_number = int(DrillingData_df["N_rigs"])
    max_Nw = int(DrillingData_df["Total_Nw"])
    Nw_field = int(DrillingData_df["Nw_predrilled"])
    p_i_ratio = float(DrillingData_df["P_I_ratio"])

    # Create dataframe with drilling schedule
    DrillingSch_df = pd.DataFrame()
    DrillingSch_df["time"] = TypeCurve_df["time"]
    DrillingSch_df["time_days"] = TypeCurve_df["time_days"]
    DrillingSch_df["Nw_field_p&i"] = DrillingSch_df.apply(
        lambda x: int(DrillingData_df["Nw_predrilled"]), axis=1
    )
    DrillingSch_df.loc[0, "Nw_field_p&i"] = "-"

    # Column with number of production and injection wells in the field in each time step
    for w in range(2, len(DrillingSch_df)):
        DrillingSch_df.loc[w, "Nw_field_p&i"] = (
            DrillingSch_df.loc[w - 1, "Nw_field_p&i"]
            + (
                math.floor(
                    (
                        (
                            DrillingSch_df.loc[w, "time_days"]
                            - DrillingSch_df.loc[w - 1, "time_days"]
                        )
                        / int(DrillingData_df["Drilling_time_ave"])
                    )
                )
            )
            * Rigs_number
        )

        if DrillingSch_df.loc[w, "Nw_field_p&i"] >= max_Nw:
            DrillingSch_df.loc[w, "Nw_field_p&i"] = max_Nw

    # Column with number of production wells only in each time step
    DrillingSch_df["Nw_field_prod"] = DrillingSch_df.apply(lambda x: 0, axis=1)

    for j in range(1, len(DrillingSch_df)):
        DrillingSch_df.loc[j, "Nw_field_prod"] = DrillingSch_df.loc[
            j, "Nw_field_p&i"
        ] / (1 + (1 / p_i_ratio))
        DrillingSch_df.loc[j, "Nw_field_prod"] = math.floor(
            DrillingSch_df.loc[j, "Nw_field_prod"]
        )
    DrillingSch_df.loc[0, "Nw_field_prod"] = "-"

    # print(DrillingSch_df)

    DrillingSch_dict["Drill_Sch_" + key.split("_")[1]] = DrillingSch_df
    # print(DrillingSch_dict)

    # Determine number of wells in the field in each time step
    # Number of wells in the field
    # DrillingSch_df['Nw_prod'] = DrillingSch_df['Nw_per_year'] / (1 + (1/DrillingSch_df['P_I_ratio']))
    # for j in range(len(DrillingSch_df)):
    #     DrillingSch_df.loc[j, 'Nw_prod'] = math.floor(DrillingSch_df.loc[j, 'Nw_prod'])
    #
    # DrillingSch_df['Nw_prod_in_field'] = DrillingSch_df['Nw_prod'].cumsum()
    # print(DrillingSch_df)

In [ ]:
    ########################################################################################################################
    ############                                                                                                ############
    ############                      Determine Potential Production of the field                               ############
    ############                                                                                                ############
    ########################################################################################################################

    # Production potential estimation
    pot_prod = pd.DataFrame(TypeCurve_df)
    for col in pot_prod.columns[2:]:
        for k in range(2, len(TypeCurve_df)):
            pot_prod.loc[k, col] = (
                pot_prod.loc[k, col] * DrillingSch_df.loc[k, "Nw_field_prod"]
            )

    # Cumulative production from production potential
    pot_prod["cum_oil_pot"] = pot_prod.apply(lambda x: 0, axis=1)
    pot_prod["cum_gas_pot"] = pot_prod.apply(lambda x: 0, axis=1)
    pot_prod["cum_water_pot"] = pot_prod.apply(lambda x: 0, axis=1)

    for i in range(2, len(pot_prod)):
        pot_prod.loc[i, "cum_oil_pot"] = (
            pot_prod.loc[i - 1, "cum_oil_pot"]
            + (pot_prod.loc[i - 1, "oil_rate"] + pot_prod.loc[i, "oil_rate"])
            * (pot_prod.loc[i, "time_days"] - pot_prod.loc[i - 1, "time_days"])
            / 2
        )

        pot_prod.loc[i, "cum_gas_pot"] = (
            pot_prod.loc[i - 1, "cum_gas_pot"]
            + (pot_prod.loc[i - 1, "gas_rate"] + pot_prod.loc[i, "gas_rate"])
            * (pot_prod.loc[i, "time_days"] - pot_prod.loc[i - 1, "time_days"])
            / 2
        )

        pot_prod.loc[i, "cum_water_pot"] = (
            pot_prod.loc[i - 1, "cum_water_pot"]
            + (pot_prod.loc[i - 1, "water_rate"] + pot_prod.loc[i, "water_rate"])
            * (pot_prod.loc[i, "time_days"] - pot_prod.loc[i - 1, "time_days"])
            * 365
            / 2
        )

    # set units of columns
    if TypeCurve_df.loc[0, "oil_rate"] == "sm3/days":
        pot_prod.loc[0, "cum_oil_pot"] = "sm3"
    else:
        pot_prod.loc[0, "cum_oil_pot"] = "stb"

    if TypeCurve_df.loc[0, "gas_rate"] == "sm3/days":
        pot_prod.loc[0, "cum_gas_pot"] = "sm3"
    else:
        pot_prod.loc[0, "cum_gas_pot"] = "scf"

    if TypeCurve_df.loc[0, "water_rate"] == "sm3/days":
        pot_prod.loc[0, "cum_water_pot"] = "sm3"
    else:
        pot_prod.loc[0, "cum_water_pot"] = "stb"

    # print(pot_prod)

    pot_prod_dict["pot_prod_" + key.split("_")[1]] = pot_prod
    # print(pot_prod_dict)

In [ ]:
    ########################################################################################################################
    ############                                                                                                ############
    ############                      Estimate Production Profile of the field                                  ############
    ############                                                                                                ############
    ########################################################################################################################

    # Production profile generation
    prod_profile = pd.DataFrame()
    prod_profile["time"] = pot_prod["time"]
    prod_profile["time_days"] = pot_prod["time_days"]
    prod_profile["field_oil_rate"] = prod_profile.apply(lambda x: Plateau_rate, axis=1)
    prod_profile["field_cum_oil_rate"] = prod_profile.apply(lambda x: 0, axis=1)
    prod_profile["field_pot_oil_rate"] = prod_profile.apply(
        lambda x: pot_prod.loc[1, "oil_rate"], axis=1
    )
    prod_profile["field_pot_gas_rate"] = prod_profile.apply(
        lambda x: pot_prod.loc[1, "gas_rate"], axis=1
    )
    prod_profile["field_pot_water_rate"] = prod_profile.apply(
        lambda x: pot_prod.loc[1, "water_rate"], axis=1
    )

    # set units of variables:
    prod_profile.loc[0, "field_oil_rate"] = pot_prod.loc[0, "oil_rate"]
    prod_profile.loc[0, "field_cum_oil_rate"] = pot_prod.loc[0, "cum_oil_pot"]
    prod_profile.loc[0, "field_pot_oil_rate"] = pot_prod.loc[0, "oil_rate"]
    prod_profile.loc[0, "field_pot_gas_rate"] = pot_prod.loc[0, "gas_rate"]
    prod_profile.loc[0, "field_pot_water_rate"] = pot_prod.loc[0, "water_rate"]

    if (
        prod_profile.loc[1, "field_pot_oil_rate"]
        < prod_profile.loc[1, "field_oil_rate"]
    ):
        prod_profile.loc[1, "field_oil_rate"] = prod_profile.loc[
            1, "field_pot_oil_rate"
        ]

    # Create new dataframe of pot_prod without row of units to be used in interpolation function
    pot_prod_no_units = pd.DataFrame()
    for ii in range(1, len(pot_prod)):
        pot_prod_no_units.loc[ii - 1, "time"] = pot_prod.loc[ii, "time"]
        pot_prod_no_units.loc[ii - 1, "time_days"] = pot_prod.loc[ii, "time_days"]
        pot_prod_no_units.loc[ii - 1, "oil_rate"] = pot_prod.loc[ii, "oil_rate"]
        pot_prod_no_units.loc[ii - 1, "gas_rate"] = pot_prod.loc[ii, "gas_rate"]
        pot_prod_no_units.loc[ii - 1, "water_rate"] = pot_prod.loc[ii, "water_rate"]
        pot_prod_no_units.loc[ii - 1, "cum_oil_pot"] = pot_prod.loc[ii, "cum_oil_pot"]
        pot_prod_no_units.loc[ii - 1, "cum_gas_pot"] = pot_prod.loc[ii, "cum_gas_pot"]
        pot_prod_no_units.loc[ii - 1, "cum_water_pot"] = pot_prod.loc[
            ii, "cum_water_pot"
        ]
    # print(pot_prod_no_units)

    for t in range(2, len(prod_profile)):
        prod_profile.loc[t, "field_cum_oil_rate"] = (
            prod_profile.loc[t - 1, "field_cum_oil_rate"]
            + (
                prod_profile.loc[t - 1, "field_oil_rate"]
                + prod_profile.loc[t, "field_oil_rate"]
            )
            * (prod_profile.loc[t, "time_days"] - prod_profile.loc[t - 1, "time_days"])
            / 2
        )

        prod_profile.loc[t, "field_pot_oil_rate"] = np.interp(
            prod_profile.loc[t, "field_cum_oil_rate"],
            pot_prod_no_units["cum_oil_pot"],
            pot_prod_no_units["oil_rate"],
        )

        prod_profile.loc[t, "field_pot_gas_rate"] = np.interp(
            prod_profile.loc[t, "field_cum_oil_rate"],
            pot_prod_no_units["cum_oil_pot"],
            pot_prod_no_units["gas_rate"],
        )

        prod_profile.loc[t, "field_pot_water_rate"] = np.interp(
            prod_profile.loc[t, "field_cum_oil_rate"],
            pot_prod_no_units["cum_oil_pot"],
            pot_prod_no_units["water_rate"],
        )

        if (
            prod_profile.loc[t, "field_pot_oil_rate"]
            < prod_profile.loc[t, "field_oil_rate"]
        ):
            prod_profile.loc[t, "field_oil_rate"] = prod_profile.loc[
                t, "field_pot_oil_rate"
            ]

    # print(prod_profile)

    prod_profile_dict["prod_profile_" + key.split("_")[1]] = prod_profile
    # print(prod_profile_dict)

In [ ]:
########################################################################################################################
############                                                                                                ############
############                               Generate and export results                                      ############
############                                                                                                ############
########################################################################################################################
# # Export results
#
# # Pandas' Excel writer using XlsxWriter as the engine.
# excel_writer = pd.ExcelWriter(r'Results\Output_0' + str(n_file) + '.xlsx', engine='xlsxwriter')
#
# # Write each dataframe to a different worksheet.
# DrillingSch_df.to_excel(excel_writer, sheet_name='Drilling_Sch', startrow=2)
# pot_prod.to_excel(excel_writer, sheet_name='Prod_Potential', startrow=2)
# prod_profile.to_excel(excel_writer, sheet_name='Prod_Profile', startrow=2)
#
# # Get the xlsxwriter object from the dataframe writer object.
# workbook = excel_writer.book
# ws_drill_sch = excel_writer.sheets['Drilling_Sch']
# ws_pot = excel_writer.sheets['Prod_Potential']
# ws_profile = excel_writer.sheets['Prod_Profile']
#
# # Write information in each sheet of the Excel file.
# ws_drill_sch.write('B1', 'Drilling Schedule Data')
# ws_pot.write('B1', 'Potential Production Data')
# ws_profile.write('B1', 'Production Profile Data')
#
# # Create charts.
# max_row = len(prod_profile)
#
# # Chart: drilling schedule
# chart_drill = workbook.add_chart({'type': 'column'})
#
# chart_drill.add_series({
#     'name': 'Nw_production',
#     'categories': ['Drilling_Sch', 4, 2, max_row + 2, 2],
#     'values': ['Drilling_Sch', 4, 4, max_row + 2, 4]
# })
#
# chart_drill.set_x_axis({'name': 'Time, ' + str(prod_profile.loc[0, 'time_days'])})
# chart_drill.set_y_axis({'name': 'Number of wells'})
#
# ws_drill_sch.insert_chart('G2', chart_drill)
#
#
# # Chart: oil production potential vs. Np
# chart_pot = workbook.add_chart({'type': 'line'})
#
# chart_pot.add_series({
#     'name': 'Oil prod. pot.',
#     'categories': ['Prod_Potential', 4, 6, max_row +2, 6],
#     'values': ['Prod_Potential', 4, 3, max_row + 2, 3]
# })
#
# chart_pot.set_x_axis({'name': 'Cum. oil prod, ' + str(prod_profile.loc[0, 'field_cum_oil_rate'])})
# chart_pot.set_y_axis({'name': 'Oil rate, ' + str(prod_profile.loc[0, 'field_pot_oil_rate'])})
#
# ws_pot.insert_chart('K2', chart_pot)
#
#
#
#
# # Chart: oil production profile vs. time
# chart_profile = workbook.add_chart({'type': 'line'})
#
# chart_profile.add_series({
#     'name': 'Oil prod. profile.',
#     'categories': ['Prod_Profile', 4, 2, max_row +2, 2],
#     'values': ['Prod_Profile', 4, 3, max_row + 2, 3]
# })
#
# chart_profile.set_x_axis({'name': 'time, ' + str(prod_profile.loc[0, 'time_days'])})
# chart_profile.set_y_axis({'name': 'Oil rate, ' + str(prod_profile.loc[0, 'field_oil_rate'])})
#
# ws_profile.insert_chart('K2', chart_profile)
#
#
#
# # Close the Pandas Excel writer and output the Excel file.
# excel_writer.save()
#
# n_file += 1

# print(TypeCurves_dict)
# print(prod_profile_parameter_dict)
# print(DrillingSch_dict)
# print(pot_prod_dict)
# print(prod_profile_dict)


xlxs_writer = pd.ExcelWriter(r"Results\Output.xlsx", engine="xlsxwriter")
workbook = xlxs_writer.book

Row = 3

# Write Type Curve input data to Excel
for key, val in TypeCurves_dict.items():
    TyCur_to_excel = val

    TyCur_to_excel.to_excel(xlxs_writer, sheet_name="TypeCurve", startrow=Row)
    ws_type_curve = xlxs_writer.sheets["TypeCurve"]
    ws_type_curve.write("B" + str(Row), key)

    Row = Row + len(TyCur_to_excel) + 3


# Write Drilling Schedule to Excel
Row2 = 3
for key, val in DrillingSch_dict.items():
    DriSch_to_excel = val

    DriSch_to_excel.to_excel(xlxs_writer, sheet_name="DrillingSch", startrow=Row2)
    ws_drill_sch = xlxs_writer.sheets["DrillingSch"]
    ws_drill_sch.write("B" + str(Row2), key)

    # Create drilling schedule chart
    chart_drill = workbook.add_chart({"type": "column"})

    chart_drill.add_series(
        {
            "name": "Nw_production",
            "categories": [
                "DrillingSch",
                Row2 + 2,
                2,
                Row2 + 2 + len(DriSch_to_excel) - 2,
                2,
            ],
            "values": [
                "DrillingSch",
                Row2 + 2,
                4,
                Row2 + 2 + len(DriSch_to_excel) - 2,
                4,
            ],
        }
    )

    chart_drill.set_x_axis(
        {"name": "Time, " + str(DriSch_to_excel.loc[0, "time_days"])}
    )
    chart_drill.set_y_axis({"name": "Number of wells"})

    ws_drill_sch.insert_chart("G" + str(Row2 + 1), chart_drill)

    Row2 = Row2 + len(DriSch_to_excel) + 3


# Write Potential data to Excel
Row3 = 3
for key, val in pot_prod_dict.items():
    potprod_to_excel = val

    potprod_to_excel.to_excel(xlxs_writer, sheet_name="Prod_Potential", startrow=Row3)
    ws_pot = xlxs_writer.sheets["Prod_Potential"]
    ws_pot.write("B" + str(Row3), key)

    # Create oil production potential vs. Np
    chart_pot = workbook.add_chart({"type": "line"})

    chart_pot.add_series(
        {
            "name": "Oil prod. pot.",
            "categories": [
                "Prod_Potential",
                Row3 + 2,
                6,
                Row3 + 2 + len(potprod_to_excel) - 2,
                6,
            ],
            "values": [
                "Prod_Potential",
                Row3 + 2,
                3,
                Row3 + 2 + len(potprod_to_excel) - 2,
                3,
            ],
        }
    )

    chart_pot.set_x_axis(
        {"name": "Cum. oil prod, " + str(potprod_to_excel.loc[0, "cum_oil_pot"])}
    )
    chart_pot.set_y_axis(
        {"name": "Oil rate, " + str(potprod_to_excel.loc[0, "oil_rate"])}
    )

    ws_pot.insert_chart("K" + str(Row3 + 1), chart_pot)

    Row3 = Row3 + len(potprod_to_excel) + 3


# Write Potential data to Excel
Row4 = 3
for key, val in prod_profile_dict.items():
    prodprof_to_excel = val

    prodprof_to_excel.to_excel(xlxs_writer, sheet_name="Prod_Profile", startrow=Row4)
    ws_profile = xlxs_writer.sheets["Prod_Profile"]
    ws_profile.write("B" + str(Row4), key)

    # Create oil production profile vs. time chart
    chart_profile = workbook.add_chart({"type": "line"})

    chart_profile.add_series(
        {
            "name": "Oil prod. profile.",
            "categories": [
                "Prod_Profile",
                Row4 + 2,
                2,
                Row4 + 2 + len(prodprof_to_excel) - 2,
                2,
            ],
            "values": [
                "Prod_Profile",
                Row4 + 2,
                3,
                Row4 + 2 + len(prodprof_to_excel) - 2,
                3,
            ],
        }
    )

    chart_profile.set_x_axis(
        {"name": "time, " + str(prodprof_to_excel.loc[0, "time_days"])}
    )
    chart_profile.set_y_axis(
        {"name": "Oil rate, " + str(prodprof_to_excel.loc[0, "field_oil_rate"])}
    )

    ws_profile.insert_chart("K" + str(Row4 + 1), chart_profile)

    Row4 = Row4 + len(prodprof_to_excel) + 3


Row5 = 3

# Write production profile parameters to Excel
for key, val in prod_profile_parameter_dict.items():
    parameters_to_excel = val

    parameters_to_excel.to_excel(
        xlxs_writer, sheet_name="Prod_Prof_Param", startrow=Row5
    )
    ws_param = xlxs_writer.sheets["Prod_Prof_Param"]
    ws_param.write("B" + str(Row5), key)

    Row5 = Row5 + len(parameters_to_excel) + 3


# # Write information in each sheet of the Excel file.
ws_type_curve.write("B1", "Type Curves")
ws_drill_sch.write("B1", "Drilling Schedule Data")
ws_pot.write("B1", "Potential Production Data")
ws_profile.write("B1", "Production Profile Data")
ws_param.write("B1", "Production Profile Parameters")


# Close the Pandas Excel writer and output the Excel file.
xlxs_writer.save()

